In [7]:
!kaggle datasets download iamsouravbanerjee/indian-food-images-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/iamsouravbanerjee/indian-food-images-dataset
License(s): other




  0%|          | 0.00/355M [00:00<?, ?B/s]
 21%|██        | 74.0M/355M [00:00<00:00, 770MB/s]
 42%|████▏     | 148M/355M [00:00<00:00, 652MB/s] 
 60%|█████▉    | 212M/355M [00:00<00:00, 567MB/s]
 76%|███████▌  | 268M/355M [00:00<00:00, 515MB/s]
 90%|████████▉ | 318M/355M [00:00<00:00, 489MB/s]
100%|██████████| 355M/355M [00:00<00:00, 496MB/s]


In [8]:
import os
os.remove("List of Indian Foods.txt")

In [10]:
os.rename("Indian Food Images", "dataset")

In [11]:
DATA_DIR = "dataset/Indian Food Images"

In [13]:
food_list = os.listdir(DATA_DIR)

In [14]:
food_list

['adhirasam',
 'aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'anarsa',
 'ariselu',
 'bandar_laddu',
 'basundi',
 'bhatura',
 'bhindi_masala',
 'biryani',
 'boondi',
 'butter_chicken',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chhena_kheeri',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chikki',
 'daal_baati_churma',
 'daal_puri',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'doodhpak',
 'double_ka_meetha',
 'dum_aloo',
 'gajar_ka_halwa',
 'gavvalu',
 'ghevar',
 'gulab_jamun',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kajjikaya',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'kofta',
 'kuzhi_paniyaram',
 'lassi',
 'ledikeni',
 'litti_chokha',
 'lyangcha',
 'maach_jhol',
 'makki_di_roti_sarson_da_saag',
 'malapua',
 'misi_roti',
 'misti_doi',
 'modak',
 'mysore_pak',
 'naan',
 'navrattan_korma',
 'palak_paneer',
 'paneer_butter_masala',
 'phirni',
 'pithe',
 'poha',
 'poornalu

In [15]:
len(food_list)

80

In [30]:
from tensorflow.keras.applications import EfficientNetV2B3
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os

In [31]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(300, 300))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [32]:
def load_dataset():
    data = []
    for food in food_list:
        food_dir = os.path.join(DATA_DIR, food)
        if os.path.isdir(food_dir):
            for img_file in os.listdir(food_dir):
                img_path = os.path.join(food_dir, img_file)
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_array = load_and_preprocess_image(img_path)
                    data.append((img_array, food))
    return data

In [33]:
data = load_dataset()

In [34]:
def get_train_test_split(data, test_size=0.2):
    np.random.shuffle(data)
    split_index = int(len(data) * (1 - test_size))
    train_data = data[:split_index]
    test_data = data[split_index:]
    return train_data, test_data

In [35]:
train_data, test_data = get_train_test_split(data)

In [36]:
X_train = []
y_train = []
for item in train_data:
    img_arr, label = item 
    X_train.append(img_arr[0])
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
    